## Más allá de las excepciones: `Result`

Hemos visto que las excepciones que emite un programa pueden alterar el flujo de trabajo del mismo. En el caso en que las excepciones no son capturadas, el programa simplemente termina, con los consiguientes perjucios para el usuario.

Vimos también que tenemos la posibilidad de emitir nuestras propias excepciones, sin utilizar las propias del sistema operativo. 

En el ejemplo previo, teníamos una función `divide` que emitía cierto tipo de excepciones de acuerdo a casos que son errores (ya sea porque se trata de realizar una operación prohibida), o casos en los cuales el caso de uso no está permitido por el sistema (como dividir por uno).  




In [2]:
exception DivideByOne of string 

let divide x y =
    match y with 
    | 0.0 -> invalidOp "Trying to divide by zero!!" 
    | 1.0 -> raise (DivideByOne "Can't believe you are trying to divide by one")
    | _ -> x/y 


Por otra parte, teníamos la función `annoyedDivide` que es quien llama a `divide` y es la encargada de capturar los posibles errores a través de excepciones:

In [3]:
let annoyedDivide x y =
    try 
        divide x y 
    with 
    | DivideByOne s -> 
        printfn "%A" (s)
        -1.0
    | :? System.InvalidOperationException as ex -> 
        printfn "Message: %A" (ex.Message)
        -1.0          

        

Qué pasaría si además de dividir dos números, tenemos que calcular la raíz cuadrada del resultado? Tendríamos que usar la función `sqrt`

In [3]:
printfn $"Sqrt 4: {sqrt 4.0}"
printfn $"Sqrt 4: {sqrt -4.0}"

Sqrt 4: 2
Sqrt 4: NaN


Pero tenemos que validar que el input de la función no sea negativo:

In [4]:
exception CannotSqrtOfNegative of string 

let squareRoot (x: float) = 
    if x < 0 then 
        raise (CannotSqrtOfNegative "Cannot compute sqrt of a negative number")
    else 
        sqrt x         

In [5]:
printfn "%A" (squareRoot 4.0)
printfn "%A" (squareRoot -4.0)

2.0


Error: FSI_0008+CannotSqrtOfNegative: CannotSqrtOfNegative "Cannot compute sqrt of a negative number"
   at FSI_0008.squareRoot(Double x)
   at <StartupCode$FSI_0009>.$FSI_0009.main@()
   at System.RuntimeMethodHandle.InvokeMethod(Object target, Void** arguments, Signature sig, Boolean isConstructor)
   at System.Reflection.MethodBaseInvoker.InvokeWithNoArgs(Object obj, BindingFlags invokeAttr)

In [6]:
let annoyedSqrt x =
    try 
        squareRoot x 
    with 
    | CannotSqrtOfNegative s -> 
        printfn "Message: %A" s
        -1 


Una vez más nos encontramos con el problema sobre qué retornar cuando se captura la excepción.

In [7]:
let x = 5.0
let y = 3.0 

sqrt (x/y)

1.2909944487358056

In [8]:
let x = 5
let y = 0 

sqrt  (annoyedDivide x y)

Message: "Trying to divide by zero!!"


NaN

In [9]:
let x = -5
let y = 3 

annoyedSqrt  (annoyedDivide x y)

Message: "Cannot compute sqrt of a negative number"


-1

In [10]:
let x = -5
let y = 3 

annoyedDivide x y
|> annoyedSqrt

Message: "Cannot compute sqrt of a negative number"


-1

### El tipo `Result`

Para solucionar los inconvenientes que presenta esta aproximación al manejo de errores, F# provee un tipo de dato a tal efecto. Observemos que tanto `divide` como `squareRoot` tienen dos salidas posibles, dependiendo de si el cálculo se puede efectuar o si hay algún error. En ningún caso uno puede estar en ambas situaciones al mismo tiempo, es decir, son situaciones _disjuntas_. Para eso están las uniones discriminadas!

El lenguaje define un tipo de unión particular, denominado `Result`:

```fsharp
type Result<'T,'TError> =
    | Ok of 'T
    | Error of 'TError
```

`Result es un tipo genérico que puede encontrarse en alguno de los dos estados posibles
- `Ok 'T` cuando la operación se pudo realizar
- `Error 'TError` cuando existe un error.

In [11]:
let divideR x y =
    match y with 
    | 0.0 -> Error "Trying to divide by zero!!" 
    | 1.0 -> Error "Can't believe you are trying to divide by one"
    | _ -> Ok (x/y) 

printfn "%A" (divideR 4 2)
printfn "%A" (divideR 4 1)
printfn "%A" (divideR 4 0)

Ok 2.0
Error "Can't believe you are trying to divide by one"
Error "Trying to divide by zero!!"


In [12]:
let squareRootR (x: float) = 
    if x < 0 then 
        "Cannot compute sqrt of a negative number" |> Error 
    else 
        sqrt x |> Ok       

In [13]:
printfn "%A" (squareRootR 4.0)
printfn "%A" (squareRootR -4.0)

Ok 2.0
Error "Cannot compute sqrt of a negative number"


#### Railway programming

Evidentemente, el uso del tipo `Result` aclara significativamente el código. Pero ahora sucede que:

```fsharp
divideR:
   x: float ->
   y: float
   -> Result<float,string>
```

mientras que 

```fsharp 
squareRootR:
   x: float
   -> Result<float,string>
```

Entonces no me es posible componer ambas funciones, dado que el output de `divideR` es de tipo `Result`, mientras que el input de `squareRootR` es un `float`.  




In [18]:
let x = 4
let y = 6 

let r = divideR x y 
        |> Result.bind squareRootR

printfn "%A" r         

Ok 0.8164965809


In [19]:
let x = 4
let y = 0 

let r = divideR x y 
        |> Result.bind squareRootR

printfn "%A" r        

Error "Trying to divide by zero!!"


In [20]:
let x = -4
let y = 4 

let r = divideR x y 
        |> Result.bind squareRootR

printfn "%A" r      

Error "Cannot compute sqrt of a negative number"


El método `bind` de `Result` se ocupa de emparejar los inputs y outputs para poder conectar ambas funciones:

```fsharp 
bind:
   binder: ('T -> Result<'U,'TError>) ->
   result: Result<'T,'TError>
        -> Result<'U,'TError>
bind f inp evaluates to match inp with Error e -> Error e | Ok x -> f x
```

Supongamos ahora que además queremos sumarle un valor al resultado de $\sqrt{x/y}$. Afortunadamente la suma es una operación menos restrictiva que las anteriores, así que no es necesario tanta ceremonia:

In [21]:
let add10 y =
    y + 10.0 

Pero ahora tenemos otro problema, porque nuestra operación completa nos da como output `Result<float,string>`, que no es el input de `add10`....

Para eso existe el método `map` de `Result`: 


In [23]:
let x = 8
let y = 2 

let r = 
    divideR x y 
    |> Result.bind squareRootR
    |> Result.map add10 

printfn "%A" r    

Ok 12.0


In [25]:
let x = 8
let y = -2 

let r = 
    divideR x y 
    |> Result.bind squareRootR
    |> Result.map add10 

printfn "%A" r    

Error "Cannot compute sqrt of a negative number"


In [26]:
let x = 8
let y = 0 

let r = 
    divideR x y 
    |> Result.bind squareRootR
    |> Result.map add10 

printfn "%A" r    

Error "Trying to divide by zero!!"


In [28]:
let x = 8
let y = 0 

let res = 
    divideR x y 
    |> Result.bind squareRootR
    |> Result.map add10 

let err = 
    res
    |> Result.mapError (fun e -> $"Hubo un error: {e}")    

printfn  "%A" err    

Error "Hubo un error: Trying to divide by zero!!"
